In [3]:
import feedparser
import requests
from bs4 import BeautifulSoup
from datetime import datetime

# Fonction pour nettoyer le HTML et obtenir le texte brut
def clean_html(html_content):
    soup = BeautifulSoup(html_content, "html.parser")
    return soup.get_text()  # Retourne le texte brut sans balises HTML

# Fonction pour convertir une date Feedparser en ISO 8601
def convert_to_iso8601(date_str):
    try:
        # Parse la date avec le format de Feedparser
        parsed_date = datetime.strptime(date_str, "%a, %d %b %Y %H:%M:%S %z")
        # Convertit en format ISO 8601
        return parsed_date.isoformat()
    except ValueError:
        return None  # Retourne None si la date est invalide ou absente

# Liste des flux RSS (Feedly et Google Alerts)
rss_feeds = [
    "http://javascript.developpez.com/index/rss",  # lien feedly
    "https://www.google.com/alerts/feeds/11440117351863688430/4739719902568766787",     # Google Alert
]

# Notion API Configuration
NOTION_API_URL = "https://api.notion.com/v1/pages"
NOTION_API_KEY = "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"  # Remplace par ton Token Notion
DATABASE_ID = "144e6de098618089bdb0cc4b71dea02f"  # Remplace par l'ID de ta base de données

# Fonction pour analyser les flux RSS
def parse_rss_feeds(feeds):
    articles = []
    for feed_url in feeds:
        print(f"Analyse du flux RSS : {feed_url}")
        feed = feedparser.parse(feed_url)
        
        # Vérifie si le flux est valide
        if 'title' in feed.feed:
            print(f"Flux trouvé : {feed.feed.title}")
        else:
            print("Erreur : Flux non valide.")
            continue
        
        for entry in feed.entries:
            article = {
                "title": entry.title,
                "link": entry.link,
                "published": entry.published if "published" in entry else "N/A",
                "summary": entry.summary if "summary" in entry else "N/A"
            }
            articles.append(article)
    
    return articles

# Générer des mots-clés depuis un résumé
def generate_keywords(summary):
    keywords = []
    if "innovation" in summary.lower():
        keywords.append("Innovation")
    if "nouveau produit" in summary.lower():
        keywords.append("Nouveau Produit")
    if "npm" in summary.lower():
        keywords.append("npm")
    if "react" in summary.lower():
        keywords.append("react")
    if "js" in summary.lower():
        keywords.append("js")
    # Ajoute d'autres mots-clés selon les critères
    return keywords

# Filtrer les articles selon les mots-clés
def filter_articles(articles):
    filtered = []
    for article in articles:
        # Nettoyer le résumé pour générer les mots-clés
        cleaned_summary = clean_html(article["summary"])
        keywords = generate_keywords(cleaned_summary)

        # Vérifier si l'un des mots-clés est "npm", "react" ou "js"
        if any(keyword in ["npm", "react", "js"] for keyword in keywords):
            article["keywords"] = keywords  # Ajouter les mots-clés à l'article
            filtered.append(article)
    return filtered

# Ajouter un article à Notion
def add_article_to_notion(article):
    # Nettoyage du résumé
    cleaned_summary = clean_html(article["summary"])

    # Conversion de la date en format ISO 8601
    iso_date = convert_to_iso8601(article["published"])

    headers = {
        "Authorization": f"Bearer {NOTION_API_KEY}",
        "Content-Type": "application/json",
        "Notion-Version": "2022-06-28"
    }

    # Préparer les données à envoyer à Notion
    data = {
        "parent": {"database_id": DATABASE_ID},
        "properties": {
            "Title": {"title": [{"text": {"content": article["title"]}}]},
            "Link": {"url": article["link"]},
            "Published": {"date": {"start": iso_date} if iso_date else None},
            "Summary": {"rich_text": [{"text": {"content": cleaned_summary}}]},  # Utilise le résumé nettoyé
            "Keywords": {"multi_select": [{"name": keyword} for keyword in article["keywords"]]}  # Ajoute les mots-clés
        }
    }

    # Envoyer les données à Notion
    response = requests.post(NOTION_API_URL, headers=headers, json=data)
    if response.status_code == 200:
        print(f"Article ajouté à Notion : {article['title']}")
    else:
        print(f"Erreur lors de l'ajout à Notion : {response.status_code}, {response.text}")

# Fonction principale
if __name__ == "__main__":
    # Étape 1 : Analyse des flux RSS
    collected_articles = parse_rss_feeds(rss_feeds)
    
    # Étape 2 : Filtrer les articles selon les mots-clés
    filtered_articles = filter_articles(collected_articles)
    
    # Étape 3 : Envoi des articles filtrés à Notion
    print(f"\nAjout des articles filtrés à Notion...")
    for article in filtered_articles:
        add_article_to_notion(article)

    print(f"\n{len(filtered_articles)} articles filtrés ajoutés à Notion.")


Analyse du flux RSS : http://javascript.developpez.com/index/rss
Flux trouvé : Flux JavaScript Developpez
Analyse du flux RSS : https://www.google.com/alerts/feeds/11440117351863688430/4739719902568766787
Flux trouvé : Google Alert - javascript

Ajout des articles filtrés à Notion...
Article ajouté à Notion : Invasion silencieuse : des centaines de bibliothèques malveillantes publiées sur NPM tentent d'installer des malwares sur les machines des développeurs pour infiltrer les entreprises
Article ajouté à Notion : Exécuter des programmes C dans le navigateur en utilisant le runtime WebAssembly, &quot;une étape majeure pour faire fonctionner n'importe quel logiciel avec WebAssembly&quot;, d'après Wasmer
Article ajouté à Notion : « L'utilisation incessante de frameworks JavaScript de pointe a contribué à rendre le Web moins accessible », d'après Easy Laptop Finder selon lequel ces derniers détruisent les performances des sites Web
Article ajouté à Notion : 70% des nouveaux paquets NPM au